<a href="https://colab.research.google.com/github/anuthereaper/PythonLibrary/blob/main/Servicebus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/servicebus/azure-servicebus/samples/async_samples
https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/servicebus/azure-servicebus/samples/sync_samples
https://docs.microsoft.com/en-us/azure/service-bus-messaging/service-bus-python-how-to-use-queues

In [3]:
pip install azure.servicebus

     |████████████████████████████████| 202 kB 4.2 MB/s 
     |████████████████████████████████| 178 kB 42.9 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 3.1 MB 34.4 MB/s 


In [ ]:
# Receive messages from service bus
from azure.servicebus import ServiceBusClient
import pandas as pd
import json
from pandas import json_normalize
CONNECTION_STR = 'Endpoint=sb://<servicebusnamespace>.servicebus.windows.net/;SharedAccessKeyName=<accesskeyname>;SharedAccessKey=<Access_Key>;EntityPath=<Service_Bus_Name>'
QUEUE_NAME = '<Service_Bus_Name>'

servicebus_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR)

with servicebus_client:
    receiver = servicebus_client.get_queue_receiver(queue_name=QUEUE_NAME)
    
    # create an Empty DataFrame object
    df = pd.DataFrame()
    msg_concat = ""
    dfs = []
    with receiver:
        received_msgs = receiver.receive_messages(max_message_count=10, max_wait_time=5)  # Receive a batch of 10 messages
        for msg in received_msgs:
            msg_dict = json.loads(str(msg))
            df2 = json_normalize(msg_dict)
            df = df.append(df2, ignore_index = True)
            receiver.complete_message(msg)
print(df)
print("Receive is done.")

In [ ]:
# Send messages to service bus
from azure.servicebus import ServiceBusMessage
# https://docs.microsoft.com/en-us/azure/service-bus-messaging/service-bus-python-how-to-use-queues
from azure.servicebus import ServiceBusClient

CONNECTION_STR = 'Endpoint=sb://<servicebusnamespace>.servicebus.windows.net/;SharedAccessKeyName=<accesskeyname>;SharedAccessKey=<Access_Key>;EntityPath=<Service_Bus_Name>'
QUEUE_NAME = '<Service_Bus_Name>'

servicebus_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR)

def send_single_message(sender):
    # create a Service Bus message
    message = ServiceBusMessage("Single Message")
    # send the message to the queue
    sender.send_messages(message)
    print("Sent a single message")

def send_a_list_of_messages(sender):
    # create a list of messages
    messages = [ServiceBusMessage("Message in list") for _ in range(5)]
    # send the list of messages to the queue
    sender.send_messages(messages)
    print("Sent a list of 5 messages")

def send_batch_message(sender):
    # create a batch of messages
    batch_message = sender.create_message_batch()
    for _ in range(10):
        try:
            # add a message to the batch
            batch_message.add_message(ServiceBusMessage("{\"Name\":\"\",\"Seri\":21000000,\"SiName\":\"\",\"As\":\"\"}"))
        except ValueError:
            # ServiceBusMessageBatch object reaches max_size.
            # New ServiceBusMessageBatch object can be created here to send more data.
            break
    # send the batch of messages to the queue
    sender.send_messages(batch_message)
    print("Sent a batch of 10 messages")

with servicebus_client:
    sender = servicebus_client.get_queue_sender(queue_name=QUEUE_NAME)
    with sender:
        send_single_message(sender)
        send_a_list_of_messages(sender)
        send_batch_message(sender)

In [ ]:
pip install names

In [ ]:
# SEND BULK LOAD MESSAGES TO SERVICE BUS
from random import randrange
import random
import names
import json
from datetime import datetime
from datetime import timedelta
from azure.servicebus import ServiceBusMessage
from azure.servicebus import ServiceBusClient
import uuid

CONNECTION_STR = 'Endpoint=sb://<servicebusnamespace>.servicebus.windows.net/;SharedAccessKeyName=<accesskeyname>;SharedAccessKey=<Access_Key>;EntityPath=<Service_Bus_Name>'
QUEUE_NAME = '<Service_Bus_Name>'

Number_of_docs = 10
d1 = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2009 4:50 AM', '%m/%d/%Y %I:%M %p')

servicebus_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR)

def send_single_message(sender,message):
    sender.send_messages(message)

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def Create_Record(item_id):
    """
    This function will return a service bus message with random data.
    """
    doc_body = {'id' : str(uuid.uuid4()),
            'employee_id' : str(id),
            'first_name' : names.get_first_name(),
            'last_name' : names.get_last_name(),
            'regn_no' : 'TEST00000000' + str(item_id),
            'start_date' : str(random_date(d1,d2)),
            'salary' : str(random.randint(12000, 2000000)),
            'inserted_at' : str(datetime.now())
            }
    json_object = json.dumps(doc_body, indent = 4) 
#   print(json_object)
    return ServiceBusMessage(bytes(json_object,'utf-8'))

sender = servicebus_client.get_queue_sender(queue_name=QUEUE_NAME)
starttime = datetime.utcnow()
print("Starting ingestion: ", starttime.strftime("%Y-%m-%d %H:%M:%S.%f"))
with servicebus_client:
    sender = servicebus_client.get_queue_sender(queue_name=QUEUE_NAME)
    with sender:
        for i in range(Number_of_docs):
            message = Create_Record(i)
            send_single_message(sender,message)
            if (i+1)%100 == 0 or (i == Number_of_docs-1):
                print(str(i+1) + " documents completed")

endtime = datetime.utcnow()
print("\nrun_sample done :" + endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
print("Time taken :" + str(endtime-starttime))
print("Number of messages :" + str(Number_of_docs))

In [12]:
# Send messages with custom and system properties (https://docs.microsoft.com/en-us/python/api/azure-servicebus/azure.servicebus.servicebusmessage?view=azure-python)
from azure.servicebus import ServiceBusMessage
import datetime
# https://docs.microsoft.com/en-us/azure/service-bus-messaging/service-bus-python-how-to-use-queues
from azure.servicebus import ServiceBusClient

CONNECTION_STR = 'Endpoint=sb://<servicebusnamespace>.servicebus.windows.net/;SharedAccessKeyName=<accesskeyname>;SharedAccessKey=<Access_Key>;EntityPath=<Service_Bus_Name>'
QUEUE_NAME = <queue_or_topic_name>

servicebus_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR)

def send_single_message(sender):
    # create a Service Bus message
    message = ServiceBusMessage("Nivis Message1",session_id="MySessionID",application_properties={'anucustomkey': 'Nivi'},time_to_live=datetime.timedelta(seconds=30),label='MyLabel')
#    message.ApplicationProperties.Add("anucustomKey", "Nivi");
    # send the message to the queue
    sender.send_messages(message)
    print("Sent a single message")

with servicebus_client:
    sender = servicebus_client.get_queue_sender(queue_name=QUEUE_NAME)
    with sender:
        send_single_message(sender)

Sent a single message
